# Business Data Analyst Test

En primer lugar, se han seguido los pasos del README para correr el docker y cargar la BBDD de postgres. A través de la consola, he consultado la BBDD y he visto que tenía 3 tablas con los nombres *message_store*, *message_store_seq_seq* y *migrations*.

A continuación, conectamos con la base de datos e importamos a dataframes de pandas las 3 tablas que hay en la base de datos.

Los primeros pasos serán para conocer los datos y de limpieza de los mismos.

## 0. Limpieza de datos

In [1]:
import pandas as pd

message_store = pd.read_sql_query('select * from message_store',con=psycopg2.connect(host="localhost",database="test", user="test", password="test"))
message_store_seq_seq = pd.read_sql_query('select * from message_store_seq_seq',con=psycopg2.connect(host="localhost",database="test", user="test", password="test"))
migrations = pd.read_sql_query('select * from migrations',con=psycopg2.connect(host="localhost",database="test", user="test", password="test"))


NameError: name 'psycopg2' is not defined

Mi intención, a continuación, es investigar qué contienen las tablas para saber qué información me va a ser útil para resolver el ejercicio.

In [ ]:
message_store.head()

In [ ]:
message_store.shape

In [ ]:
message_store_seq_seq.head()

In [ ]:
display(migrations)

En las líneas superiores vemos que:

* La única tabla con datos útiles es la tabla *message_store*.
* La tabla message_store contiene muchos datos que, a priori, parecen que no aportan información al ejercico.

Lo primero que me llama la atención es la columna *payload* que contiene un diccionario. Muestro una de las filas al azar:

In [ ]:
message_store.payload[125250]

Finalmente veo que en dicha columna parece estar gran parte de la información que necesito. Paso a copiar el dataframe a una variable que llamo **df** en el que empiezo a limpiar los datos.

In [ ]:
df = message_store.copy()

Aparentemente muchos de los datos contienen información del sistema, que no es útil para las cuestiones del ejercicio y que además están repetidas.

Hago *value_counts* para asegurar que no contienen información útil y que puedo prescindir de dichas columnas.

In [ ]:
print(df.type.value_counts())
print(df.recipient.value_counts())
print(df.sender.value_counts())
print(df.publishable.value_counts())
print(df.published.value_counts())
print(df.publish_retries.value_counts())
print(df.publish_error.value_counts())

Elimino todas estas columnas y las columnas que no aportan información. Me quedo solo con *occurred_on* y *payload*

In [ ]:
df = df[["occurred_on", "payload"]].copy()

In [ ]:
df.head()

Para hacer más manejable la información, convierto el diccionario en columnas del dataframe.

In [ ]:
df = pd.concat([df['occurred_on'], df['payload'].apply(pd.Series)], axis=1)

In [ ]:
df.head()

Volvemos a limpiar. Eliminamos id y comprobamos que hay información que no aporta nada al ejercicio y podemos eliminar. Comprobamos además que la columna *trackedAt* contiene la misma información que *lastRideAt*.

In [ ]:
df["city"].value_counts()

In [ ]:
df["client"].value_counts()

In [ ]:
df["type"].value_counts()

In [ ]:
df["driverTask"].value_counts()

In [ ]:
df["trackedAt"].value_counts()

In [ ]:
df["lastRideAt"].value_counts()

Finalmente las columnas que a priori nos pueden ser útiles son las siguientes, que alfamecenamos en un dataframe limpio de trabajo que llamo **dfw**:

In [ ]:
dfw = df[["occurred_on","battery","latitude","longitude","trackedAt"]].copy()

In [ ]:
dfw.dtypes

In [ ]:
dfw.head()

A continuación, creo un par de dataframes auxiliares que contienen las filas con datos de día y datos de noche respectivamente.

Para el ejercicio he considerado que la noche empieza a las 21 y acaba a las 7 (inclusives), si bien se podrían considerar otras franjas según la necesidad.

In [ ]:
df_night = dfw[(dfw["occurred_on"].dt.hour >= 21) | (dfw["occurred_on"].dt.hour <= 7)]
df_day = dfw[(dfw["occurred_on"].dt.hour < 21) | (dfw["occurred_on"].dt.hour > 7)]

Compruebo que la selección está bien realizada:

In [ ]:
df_night["occurred_on"].dt.hour.value_counts()

## 1. What's the battery average of the e-vehicles during the night?

In [ ]:
df_night.battery.mean()

La batería media durante la noche, según el rango horario seleccionado, es de **50,13%**.

## 2. What's the battery average of the e-vehicles during the day?

In [ ]:
df_day.battery.mean()

La batería media durante el día, según el rango horario seleccionado, es también de **50,13%**.

## 3. If e-vehicles must be picked-up to charge their battery when the level is lower than 40, what's the time range when most e-vehicles need to be recharged?

Por comodidad he creado un par de columnas nuevas en el df:

* Una con la hora
* Otra que llamo *need_charge* que valdrá 1 si necesita carga (la batería es menor que 40) y 0 si no necesita ser recogida (la carga es del 40% o superior)

In [ ]:
dfw["hour"] = dfw["occurred_on"].dt.hour

In [ ]:
dfw["need_charge"] = dfw.battery.apply(lambda x: 1 if x <40 else 0)

In [ ]:
dfw.need_charge.value_counts()

Finalmente, a través de un groupby agrupo por horas, cuento los patines que necesitan carga y los ordeno de mayor a menor según el número de patinetes.

**Se observa que el rango que más patinetes necesitan de recarga es el de 11 a 12 del mediodía.**

Sin embargo, si elegimos rangos más amplios, se pueden sacar las siguientes conclusiones:

* La noche en general necesita de más carga que el día (de 0:00 a 5 especialmente).
* En el rango horario de 8:00 a 11:00 hay un pico de necesidades de recarga (con un descenso a las 9:00).
        
El gráfico que se muestra a continuación no no es todo lo "visual" que se desearía, ya que las columnas tienen una altura muy similar. Pero con algo de atención, se observan fácilmente las conclusiones mencionadas anteriormente.

In [ ]:
dfw[dfw.need_charge == 1].groupby("hour").need_charge.count().reset_index(name='count')\
                                                .sort_values(['count'], ascending=False)

In [ ]:
import matplotlib.pyplot as plt

need_battery = dfw[dfw.need_charge == 1].groupby("hour").need_charge.count().reset_index(name='count')\
                                                .sort_values(['hour'], ascending=True)
fig, ax = plt.subplots(figsize=(8, 6))

plt.bar(need_battery.hour.tolist(), need_battery["count"].tolist())
# plt.xticks(x, ('Bill', 'Fred', 'Mary', 'Sue'))
plt.show()

## If e-vehicles must be picked-up to charge their battery when the level is lower than 40, what's the geozone with greater density of e-vehicles to be charged?

Para representar geográficamente los datos he decidido utilizar folium, una librería de python que usa de fondo leaflet.js, una librería de javascript para construir mapas interactivos.

Lo primero que hago, es agrupar los datos por latitud y longitud contando cuántos vehículos hay en cada coordenada que tengan una batería menor al 40%.

A continuación, para que la representación sea más visual, normalizo los datos de la columna *count*, restando la media y dividiendo el resultado por la desviación estándar.

He hecho dos representaciones:

* La primera de ellas he creado un rango de radios: A mayor número de vehículos que necesitan carga, mayor es el radio. Además, los 5 puntos que más carga requieren, los he puesto en rojo, para diferenciarlos de los demás que están en azul.

* La segunda representación es un mapa de calor creado con el plugin heatmap de folium.


In [ ]:
coordinates = dfw[dfw.need_charge == 1].groupby(["latitude", "longitude"]).need_charge.count()\
                                .reset_index(name='count').sort_values(['count'], ascending=False)
coordinates.head()

In [ ]:
coordinates["count_aux"] = coordinates["count"].copy()

coordinates["count"] = coordinates["count"].apply(lambda x: (x-coordinates["count"].mean())/coordinates["count"].std())

In [ ]:
import folium
import statistics

lat = list(coordinates.latitude)
lon = list(coordinates.longitude)
count = list(coordinates["count"])

map = folium.Map(location=[df_charge.latitude.mean(), df_charge.longitude.mean()], zoom_start=12)

for e in range(len(lat)):
    latitude = lat[e]
    longitude = lon[e]
    
    if count[e] < 0.2:
        radio = 150
    elif count[e] <0.4:
        radio = 200
    elif count[e] < 0.6:
        radio = 250
    elif count[e] < 0.8:
        radio = 300
    else:
        radio = 400
    
    if e in range(5):
        folium.Circle([latitude, longitude],
                        radius=radio,
                        color = "red",
                        fill_color= '#f03',
                        fill=True
                       ).add_to(map)
    else:
        folium.Circle([latitude, longitude],
                        radius=radio,
                        fill=True
                       ).add_to(map)
map

In [ ]:
from folium.plugins import HeatMap

def generateBaseMap(default_location=[dfw.latitude.mean(), dfw.longitude.mean()], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()
HeatMap(data=coordinates[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=15, max_zoom=13).add_to(base_map)
base_map

## Could you display on a heat map the density of e-vehicles in a timeline? Basically, we should be able to see how the density varies on the map as we change the timestamp.

En este punto me he encontrado un problema. Folium permite realizar esto con el plugin *HeatMapWithTime*, sin embargo, parece que no funciona muy bien. 

No he podido ponerlo en marcha, pero es que tampoco puedo ver [los ejemplos de su página web](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/HeatMapWithTime.ipynb).

Investigando por internet he visto que da fallos si tienes activado adblocks. He probado en varios navegadores y en varios ordenadores y no he conseguido ni siquiera ver el ejemplo, así que finalmente decidí realizarlo de otra manera.

El código que utilicé es el siguiente.

Los datos deben estar en una lista:

In [ ]:
coordinates_time = dfw.groupby(["occurred_on", "latitude", "longitude"]).need_charge.count().reset_index(name='count')\
                                                .sort_values(['occurred_on'], ascending=True)

coordinates_time["count_aux"] = coordinates_time["count"].copy()
coordinates_time.head()

In [ ]:
df_hour_list = []
for hour in coordinates_time["occurred_on"].dt.hour.sort_values().unique():
    df_hour_list.append(coordinates_time.loc[coordinates_time["occurred_on"].dt.hour == hour, ['latitude', 'longitude']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())

In [ ]:
from folium.plugins import HeatMapWithTime

base_map2 = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour_list, radius=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, auto_play=True, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2)
base_map2

Por ellos, he decidio realizar este ejercicio con leaflet. Para ello he creado un pequeño documento html que contiene el script de Javascript para mostrar el mapa en leaflet.

Para realizarlo, utilizo el plugin de leaflet **Leaflet.TimeDimension**.

Lo primero que necesito es una nested list que contenga en el primer elemento el timestamp (en ms, que es lo que requiere el plugin) y en el segundo, una lista con todas las coordenadas donde hay vehículos a esa hora. Además, también recojo los pesos, es decir, cuántos vehículos hay en cada coordenada a cada hora.

En primer lugar creo una nested list **con todos los datos** que contengan: el timestamp en ms, latitud, longitud y el peso.

In [ ]:
from datetime import datetime

coordinates_time.occurred_on = coordinates_time.occurred_on.apply(lambda x: x.timestamp()*1000)

times = coordinates_time.values.tolist()
times.sort(key = lambda x: x[0]) 
times

Creamos la nested list que queremos en la variable *date*.

In [ ]:
date = []
date_aux = []
weight = []
for time in times:
    if not time[0] in date_aux:
        date_aux.append(time[0])
        date.append([time[0]])
        date[len(date)-1].append([[time[1],time[2]]])
    else:
        date[len(date)-1][1].append([time[1],time[2]])
    weight.append(time[3])

# Convertimos weigth a entero
weight = [int(i) for i in weight]
date

Por último, creamos el documento html.

Se puede construir con el plugin TimeDimension un script que te dibuje el mapa de calor como hacía folium. Sin embargo, no he tenido el tiempo suficiente como para saber cómo.

Por el momento, presento un mapa de calor por cuadrículas donde cada una de ellas es más opaco, si hay más concentración de vehículos, o menos, si la concentración es más baja.

El máximo de vehículos a la misma hora y en el mismo lugar es de 7, que es cuando la cuadrícula es totalmente opaca. No obstante, no estaría mal una leyenda.

Seguiré trabajando en ello (en la leyenda y el heatmap).

El código de acontinuación, crea el archivo html. Si se corre la celda del notebook, se abré directamente la página en el navegador. También se puede descargar el repositorio y abrir el archivo *maps.html* que se encuentra en el directorio *html.



In [ ]:
import webbrowser

html = """
    <html>
    <head>
        <style>
        #mapid { height: 180px; }
        </style>
        <title>Leaflet Heatmap with time</title>        
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css" />
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.control.min.css" />
    <body>
    <header>
    
    </header>
    <div id="mapid" style="width: 100%; height: 100%;"></div>
    <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.js"></script>
    <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/iso8601-js-period@0.2.1/iso8601.min.js"></script>
    <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.min.js"></script>

    <script>
    
    L.TimeDimension.Layer.GeoJson.GeometryCollection = L.TimeDimension.Layer.GeoJson.extend({

    // Do not modify features. Just return the feature if it intersects
    // the time interval    
    _getFeatureBetweenDates: function(feature, minTime, maxTime) {
        var featureStringTimes = this._getFeatureTimes(feature);
        if (featureStringTimes.length == 0) {
            return feature;
        }
        var featureTimes = [];
        for (var i = 0, l = featureStringTimes.length; i < l; i++) {
            var time = featureStringTimes[i]
            if (typeof time == 'string' || time instanceof String) {
                time = Date.parse(time.trim());
            }
            featureTimes.push(time);
        }

        if (featureTimes[0] > maxTime || featureTimes[l - 1] < minTime) {
            return null;
        }
        return feature;
    },

});

L.timeDimension.layer.geoJson.geometryCollection = function(layer, options) {
    return new L.TimeDimension.Layer.GeoJson.GeometryCollection(layer, options);
};

    var map = L.map('mapid', {
    timeDimension: true,
    timeDimensionOptions: {
        timeInterval: "2019-11-18/2019-12-25",
        period: "PT1H"
    },
    timeDimensionControl: true,
    timeDimensionControlOptions:{
        timeSteps: 24
    }
}).setView([""" + str(df_charge.latitude.mean()) + """, """ + str(df_charge.longitude.mean()) + """], 13);


function getCommonBaseLayers(map){
    L.tileLayer('http://{s}.tile.osm.org/{z}/{x}/{y}.png').addTo(map);

}

// Create and add a TimeDimension Layer to the map
var geoJsonFeatures = {
  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
    "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    },
  },
  "features": [ """

contador = 1
for timestamp in date:
    html += """{
        "type": "Feature",
        "properties": {
          "time": """ + str(timestamp[0]) + """,
          "id": """ + str(contador) + """,
          "opacity":  """ + str(1 if weight[contador-1] == 7 else \
                                0.92 if weight[contador-1] == 6 else \
                                0.84 if weight[contador-1] == 5 else \
                                0.76 if weight[contador-1] == 4 else \
                                0.68 if weight[contador-1] == 3 else \
                                0.6 if weight[contador-1] == 2 else \
                                0.5 ) + """
        },
        "geometry": {
          "type": "GeometryCollection",
          "geometries": [ """
    
    for coord in timestamp[1]:
        html += """{
            "type": "Polygon",
            "coordinates": [[
        [""" + str(coord[1]-0.005) + """, """ + str(coord[0]-0.005) + """],
        [""" + str(coord[1]+0.005) + """, """ + str(coord[0]-0.005) + """],
        [""" + str(coord[1]+0.005) + """, """ + str(coord[0]+0.005) + """],
        [""" + str(coord[1]-0.005) + """, """ + str(coord[0]+0.005) + """],
        [""" + str(coord[1]-0.005) + """, """ + str(coord[0]-0.005) + """]
        ]]
        },"""
    
    contador += 1
    
    html += """]
        }
      },"""

html += """]
};


var geoJsonLayer = L.geoJson(geoJsonFeatures, {
  style: function(feature) {    
    return {
      "color": "#FF0000",
      "opacity": 0,
      "fillOpacity": feature.properties.opacity
    };
  }
});



map.fitBounds(geoJsonLayer.getBounds());

var geoJsonTimeLayer = L.timeDimension.layer.geoJson.geometryCollection(geoJsonLayer, {
  updateTimeDimension: true,
  updateTimeDimensionMode: 'replace',
  duration: 'PT30M',
});


geoJsonTimeLayer.addTo(map);

var baseLayers = getCommonBaseLayers(map);

</script>

    <footer>
        <p>Alfonso Román Bonachera</p>
    </footer>
    </body>
    </html>
    """

with open("html/maps.html", "w") as file:
    file.write(html)
    
webbrowser.open("html/maps.html")